# AB Testi ile BiddingYöntemlerinin Dönüşümünün Karşılaştırılması

### AB Testi Nedir ?  
* İki farklı versiyonun (A ve B) performansını istatistiksel olarak karşılaştırarak, hangisinin daha etkili olduğunu belirlemeye yarayan deney yöntemidir.

### Bidding Yöntemleri: 
* Dijital pazarlama ve reklam kampanyalarında, belirlenen hedeflere göre reklamın ne kadar ödeneceğini ve hangi kitleye gösterileceğini optimize etmeye yönelik teklif stratejileridir.

### A/B Testi ile Bidding Yöntemleri: 
* Farklı teklif (bidding) stratejilerinin performansını karşılaştırıp en yüksek dönüşüm veya en düşük maliyet gibi hedeflere ulaşmada en etkili stratejiyi seçmeye yardımcı olan yöntemler bütünüdür.

## İş Problemi
* Bir firma, kısa süre önce mevcut "maximum bidding" adı verilen teklif verme türüne alternatif olarak yeni bir teklif türü olan "average bidding"’i tanıttı.

* Müşterilerimizden biri olan X şirketi, bu yeni özelliği test etmeye karar verdi ve average bidding'in maximum bidding'den daha fazla dönüşüm getirip getirmediğini anlamak için bir A/B testi yapmak istiyor.

* A/B testi 1 aydır devam ediyor ve X şirketi şimdi sizden bu A/B testinin sonuçlarını analiz etmenizi bekliyor. X şirketi için nihai başarı ölçütü Purchase'dır. Bu nedenle, istatistiksel testler için Purchase metriğine odaklanılmalıdır.

## Veri Seti Hikayesi
* Bir firmanın web site bilgilerini içeren bu veri setinde kullanıcıların gördükleri ve tıkladıkları reklam sayıları gibi bilgilerin yanı sıra buradan gelen kazanç bilgileri yer almaktadır. Kontrol ve Test grubu olmak üzere iki ayrı veri seti vardır. Kontrol grubuna Maximum Bidding, test grubuna Average Bidding uygulanmıştır.

## Değişkenler
* Impression: Reklam görüntüleme sayısı
* Click: Görüntülenen reklama tıklama sayısı
* Purchase: Tıklanan reklamlar sonrası satın alınan ürün sayısı
* Earning: Satın alınan ürünler sonrası elde edilen kazanç

In [4]:
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import shapiro, levene, ttest_ind
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [7]:
dataframe_control = pd.read_excel("/ab_testing.xlsx", sheet_name="Control Group")
dataframe_test = pd.read_excel("/ab_testing.xlsx", sheet_name="Test Group")

df_control = dataframe_control.copy()
df_test = dataframe_test.copy()

## 1) Veriyi Anlama ve Hazırlama

### Veri setimize basit bir fonksiyon ile hızlıca bakalım.

In [11]:
import matplotlib.pyplot as plt

def check_df(dataframe, head=5):
    print("##################### Shape #####################")
    print(dataframe.shape)
    print("##################### Types #####################")
    print(dataframe.dtypes)
    print("##################### Head #####################")
    print(dataframe.head())
    print("##################### Tail #####################")
    print(dataframe.tail())
    print("##################### NA #####################")
    print(dataframe.isnull().sum())
    print("##################### Quantiles #####################")
    print(dataframe.quantile([0, 0.05, 0.50, 0.95, 0.99, 1]).T)
    
check_df(df_control)
print("********************************************************************************************************")
check_df(df_test)

##################### Shape #####################
(40, 4)
##################### Types #####################
Impression    float64
Click         float64
Purchase      float64
Earning       float64
dtype: object
##################### Head #####################
    Impression      Click  Purchase    Earning
0  82529.45927 6090.07732 665.21125 2311.27714
1  98050.45193 3382.86179 315.08489 1742.80686
2  82696.02355 4167.96575 458.08374 1797.82745
3 109914.40040 4910.88224 487.09077 1696.22918
4 108457.76263 5987.65581 441.03405 1543.72018
##################### Tail #####################
     Impression      Click  Purchase    Earning
35 132064.21900 3747.15754 551.07241 2256.97559
36  86409.94180 4608.25621 345.04603 1781.35769
37 123678.93423 3649.07379 476.16813 2187.72122
38 101997.49410 4736.35337 474.61354 2254.56383
39 121085.88122 4285.17861 590.40602 1289.30895
##################### NA #####################
Impression    0
Click         0
Purchase      0
Earning       0
dtype: int6

### Her iki df'ide birleştirelim.

In [12]:
df_control["group"] = "control"
df_test["group"] = "test"

df = pd.concat([df_control,df_test], axis=0,ignore_index=False)
print(df.head())
print(df.tail())

    Impression      Click  Purchase    Earning    group
0  82529.45927 6090.07732 665.21125 2311.27714  control
1  98050.45193 3382.86179 315.08489 1742.80686  control
2  82696.02355 4167.96575 458.08374 1797.82745  control
3 109914.40040 4910.88224 487.09077 1696.22918  control
4 108457.76263 5987.65581 441.03405 1543.72018  control
     Impression      Click  Purchase    Earning group
35  79234.91193 6002.21358 382.04712 2277.86398  test
36 130702.23941 3626.32007 449.82459 2530.84133  test
37 116481.87337 4702.78247 472.45373 2597.91763  test
38  79033.83492 4495.42818 425.35910 2595.85788  test
39 102257.45409 4800.06832 521.31073 2967.51839  test


## 2) Hipotezi oluşturalım.

# H0 : M1 = M2 (Kontrol grubu ve test grubu satın alma ortalamaları arasında fark yoktur.)
# H1 : M1!= M2 (Kontrol grubu ve test grubu satın alma ortalamaları arasında fark vardır.)


### Adım 1: Hipotez testi yapılmadan önce varsayım kontrollerini yapınız.

### 1) Normallik Varsayımı 
* H0: Normal dağılım varsayımı sağlanmaktadır.
* H1: Normal dağılım varsayımı sağlanmamaktadır

In [14]:
test_stat, pvalue = shapiro(df.loc[df["group"] == "control", "Purchase"])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = 0.9773, p-value = 0.5891


* p-value=0.5891
* HO reddedilemez. Control grubunun değerleri normal dağılım varsayımını sağlamaktadır.


### 2) Varyans Homojenliği Varsayımı
* H0: Varyanslar homojendir.
* H1: Varyanslar homojen değildir.

In [15]:
test_stat, pvalue = levene(df.loc[df["group"] == "control", "Purchase"],
                           df.loc[df["group"] == "test", "Purchase"])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = 2.6393, p-value = 0.1083


* p-value=0.1083
* HO reddedilemez. Control ve Test grubunun değerleri varyans homejenliği varsayımını sağlamaktadır.
* Varyanslar Homojendir.

### Adım 2: Normallik Varsayımı ve Varyans Homojenliği sonuçlarına göre uygun testi seçilmeli


### Varsayımlar sağlandığı için bağımsız iki örneklem t testi (parametrik test) yapılmaktadır.
* H0: M1 = M2 (Kontrol grubu ve test grubu satın alma ortalamaları arasında ist. ol.anl.fark yoktur.)
* H1: M1 != M2 (Kontrol grubu ve test grubu satın alma ortalamaları arasında ist. ol.anl.fark vardır)
* p<0.05 HO RED , p>0.05 HO REDDEDİLEMEZ

In [16]:
test_stat, pvalue = ttest_ind(df.loc[df["group"] == "control", "Purchase"],
                              df.loc[df["group"] == "test", "Purchase"],
                              equal_var=True)

print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = -0.9416, p-value = 0.3493


* p-value = 0.3493
* p > 0.05 olduğundan, H0 REDDEDİLEMEZ.
* HO reddedilemez. Kontrol ve test grubu satın alma ortalamaları arasında istatistiksel olarak anlamlı farklılık yoktur.